<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/leased_items_checkouts_by_branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHPL - Collection Analysis - **Leased Items**

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

Data sources / Documentation:

* https://ilsweb.cincinnatilibrary.org/collection-analysis/

In [1]:
#@title
import pandas as pd
import numpy as np
import altair as alt
# if we want to save output as .png use this
# NOTE: use !pip install -U altair_saver above to download / install first
# from altair_saver import save
from urllib.parse import urlencode
import requests

In [2]:
#@title
# engine = create_engine('sqlite:///file:current.db?mode=rw&uri=true')

# instead of the local db, we'll use the current_collection data set instead ...
# NOTE: the queries change slighly from the local db
base_url = 'https://collection-analysis.cincy.pl/current_collection'

In [3]:
#@title
sql = """\
-- find the ciculation of leased items by stat group of branch
select
  strftime('%Y-%m', transaction_day) as circ_month,
  stat_group_branch_name,
  -- bib_record_num,
  count(*) as count_checkouts
from
  circ_leased_items
where
  op_code = 'o'
group by
  1,
  2
order by
  1,
  2
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        '_shape': 'array'
      }
    ).content
)

# df.head()

## Find Aggregated Check-out Circulations For Leased Items by Branch and Month 

(previous 180 days)

In [6]:
#@title
# color="#0090bd"
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'circ_month',
        axis=alt.Axis(title='circulation month', labelAngle=-70),
        # sort='-y'
    ),
    y=alt.Y(
        'count_checkouts:Q',
        title = 'count total leased item checkouts',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'stat_group_branch_name',
        legend=None
    ),
    tooltip=['stat_group_branch_name', 'circ_month', 'count_checkouts']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=300,
).facet(
    facet='stat_group_branch_name',
    columns=6
)

chart.display()

alt.FacetChart(...)